In [1]:
import torch
from sklearn.datasets import fetch_openml

from neural_blueprints.utils import Trainer, infer_types
from neural_blueprints.config.architectures import TabularBERTConfig
from neural_blueprints.config.utils import TrainerConfig
from neural_blueprints.architectures import TabularBERT
from neural_blueprints.datasets import MaskedTabularDataset, TabularSingleLabelDataset
from neural_blueprints.preprocess import TabularPreprocessor

import logging
logging.basicConfig(
    level=logging.DEBUG,  # or DEBUG if you want even more detail
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s"
)

In [2]:
data = fetch_openml(name="adult", version=2, as_frame=True)
X = data.data
y = data.target

data = X.copy()
data['income'] = y

dtypes = infer_types(data)
data = data.astype(dtypes)
data.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,NaN,103497,Some-college,10,Never-married,NaN,Own-child,White,Female,0,0,30,United-States,<=50K


In [3]:
preprocessor = TabularPreprocessor()
data, discrete_features, continuous_features = preprocessor.run(data)

2025-12-28 15:11:28,213 - neural_blueprints.preprocess.tabular_preprocess - INFO - Identified 10 discrete features: ['workclass', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country', 'income']
2025-12-28 15:11:28,214 - neural_blueprints.preprocess.tabular_preprocess - INFO - Identified 5 continuous features: ['age', 'fnlwgt', 'capital-gain', 'capital-loss', 'hours-per-week']


### Income Inference Accuracy

In [4]:
dataset = TabularSingleLabelDataset(
    data=data,
    label_column='income',              # Specify the label column for single-label classification
    discrete_features=discrete_features,
    continuous_features=continuous_features
)

train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

In [5]:
bert_config = TabularBERTConfig(
    input_cardinalities = dataset.cardinalities,
    output_cardinalities = [2],
    latent_dim = 16,
    encoder_layers=8,
    dropout_p = 0.2,
    normalization = "batchnorm1d",
    activation = "gelu",
    final_activation = None
)

model = TabularBERT(bert_config)
model.blueprint()

TabularBERT(
  (input_projection): TabularInputProjection(
    (input_projections): ModuleList(
      (0): FeedForwardNetwork(
        (network): Sequential(
          (0): DenseLayer(
            (layer): Sequential(
              (0): Linear(in_features=1, out_features=128, bias=True)
              (1): NormalizationLayer(
                (network): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              )
              (2): GELU(approximate='none')
              (3): DropoutLayer(
                (dropout): Dropout(p=0.2, inplace=False)
              )
            )
          )
          (1): DenseLayer(
            (layer): Sequential(
              (0): Linear(in_features=128, out_features=64, bias=True)
              (1): NormalizationLayer(
                (network): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              )
              (2): GELU(approximate='none')
              (3): DropoutLayer(
   

TabularBERTConfig(input_cardinalities=[1, 9, 1, 16, 16, 7, 15, 6, 5, 2, 1, 1, 1, 42], output_cardinalities=[2], latent_dim=16, encoder_layers=8, dropout_p=0.2, normalization='batchnorm1d', activation='gelu', final_activation=None)

In [6]:
trainer = Trainer(
    model=model,
    config=TrainerConfig(
        criterion='cross_entropy',
        optimizer='adam',
        early_stopping_patience=2,
        learning_rate=1e-3,
        weight_decay=1e-5,
        save_weights_path="./models/bert_adult.pth",
        batch_size=128
    )
)
trainer.train(train_dataset, val_dataset, epochs=10, visualize=True)

2025-12-28 15:11:28,713 - neural_blueprints.utils.trainer - INFO - Trainer initialized on device: cpu


Directory ./models already exists. Existing weights are overwritten.


Training Epochs:  10%|█         | 1/10 [00:10<01:32, 10.23s/epoch]

Epoch 1/10, Training Loss: 0.7464, Validation Loss: 0.6923


Training Epochs:  20%|██        | 2/10 [00:20<01:21, 10.18s/epoch]

...

Training Epochs:  70%|███████   | 7/10 [01:11<00:30, 10.27s/epoch]

Epoch 7/10, Training Loss: 0.6809, Validation Loss: 0.6739


Training Epochs:  80%|████████  | 8/10 [01:22<00:20, 10.27s/epoch]

Epoch 8/10, Training Loss: 0.6809, Validation Loss: 0.6797


2025-12-28 15:13:01,140 - neural_blueprints.utils.trainer - INFO - No improvement in validation loss for 2 consecutive epochs. Early stopping at epoch 9.
Training Epochs:  80%|████████  | 8/10 [01:32<00:23, 11.55s/epoch]
2025-12-28 15:13:01,141 - neural_blueprints.utils.trainer - INFO - Training completed in 92.43 seconds.
2025-12-28 15:13:01,141 - neural_blueprints.utils.trainer - INFO - Best validation loss: 6.7394e-01


In [7]:
trainer.predict(val_dataset)

2025-12-28 15:13:01,555 - neural_blueprints.utils.trainer - INFO - Inference completed in 0.19 seconds.


Predictions: tensor([1, 1, 1, 1, 1]), 
 Ground Truth: tensor([1, 1, 1, 1, 1])
Prediction Accuracy: 0.8565


np.float64(0.8564994882292732)

### Masked Dataset Inference Accuracy

In [8]:
# Create dataset
dataset = MaskedTabularDataset(
    data, 
    discrete_features, 
    continuous_features,
    mask_prob=0.35
)

train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

In [9]:
bert_config = TabularBERTConfig(
    input_cardinalities = dataset.cardinalities,   
    latent_dim = 16,
    encoder_layers=8,
    dropout_p = 0.2,
    normalization = "batchnorm1d",
    activation = "gelu",
    final_activation = None
)

model = TabularBERT(bert_config)
model.blueprint()

TabularBERT(
  (input_projection): TabularInputProjection(
    (input_projections): ModuleList(
      (0): FeedForwardNetwork(
        (network): Sequential(
          (0): DenseLayer(
            (layer): Sequential(
              (0): Linear(in_features=1, out_features=128, bias=True)
              (1): NormalizationLayer(
                (network): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              )
              (2): GELU(approximate='none')
              (3): DropoutLayer(
                (dropout): Dropout(p=0.2, inplace=False)
              )
            )
          )
          (1): DenseLayer(
            (layer): Sequential(
              (0): Linear(in_features=128, out_features=64, bias=True)
              (1): NormalizationLayer(
                (network): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              )
              (2): GELU(approximate='none')
              (3): DropoutLayer(
   

TabularBERTConfig(input_cardinalities=[1, 9, 1, 16, 16, 7, 15, 6, 5, 2, 1, 1, 1, 42, 2], output_cardinalities=[1, 9, 1, 16, 16, 7, 15, 6, 5, 2, 1, 1, 1, 42, 2], latent_dim=16, encoder_layers=8, dropout_p=0.2, normalization='batchnorm1d', activation='gelu', final_activation=None)

In [10]:
trainer = Trainer(
    model=model,
    config=TrainerConfig(
        criterion='masked_reconstruction',
        optimizer='adam',
        early_stopping_patience=2,
        learning_rate=1e-3,
        weight_decay=1e-5,
        save_weights_path="./models/bert_adult.pth",
        batch_size=128
    )
)
trainer.train(train_dataset, val_dataset, epochs=10, visualize=True)

2025-12-28 15:13:01,614 - neural_blueprints.utils.trainer - INFO - Trainer initialized on device: cpu


Directory ./models already exists. Existing weights are overwritten.


Training Epochs:  10%|█         | 1/10 [00:27<04:10, 27.83s/epoch]

Epoch 1/10, Training Loss: 6.4697, Validation Loss: 6.2572


Training Epochs:  20%|██        | 2/10 [00:55<03:40, 27.61s/epoch]

...

Epoch 8/10, Training Loss: 6.0763, Validation Loss: 6.0404


Training Epochs:  90%|█████████ | 9/10 [04:07<00:27, 27.54s/epoch]

Epoch 9/10, Training Loss: 6.0607, Validation Loss: 6.0270


Training Epochs: 100%|██████████| 10/10 [04:35<00:00, 27.53s/epoch]
2025-12-28 15:17:36,889 - neural_blueprints.utils.trainer - INFO - Training completed in 275.27 seconds.
2025-12-28 15:17:36,889 - neural_blueprints.utils.trainer - INFO - Best validation loss: 6.0183e+00


Epoch 10/10, Training Loss: 6.0445, Validation Loss: 6.0183


In [11]:
trainer.predict(val_dataset)

2025-12-28 15:17:37,600 - neural_blueprints.utils.predict - WARNING - Warning: Only one class predicted for this feature.
2025-12-28 15:17:37,603 - neural_blueprints.utils.predict - WARNING - Warning: Only one class predicted for this feature.
2025-12-28 15:17:37,604 - neural_blueprints.utils.predict - WARNING - Warning: Only one class predicted for this feature.
2025-12-28 15:17:37,605 - neural_blueprints.utils.trainer - INFO - Inference completed in 0.65 seconds.


Feature Column 0:
Predicted attribute values: [0.1340441  0.32701483 0.3520194  0.38549593 0.24190909]
True attribute values: [0.04109589 0.49315068 0.39726028 0.34246576 0.3561644 ]
Accuracy: 0.2328

Feature Column 1:
Predicted attribute values: [4 4 4 4 4]
True attribute values: [4. 4. 4. 4. 7.]
Accuracy: 0.7009

Feature Column 2:
Predicted attribute values: [0.11909806 0.11099366 0.11901285 0.11419641 0.1179693 ]
True attribute values: [0.01793501 0.08518079 0.06797103 0.04228291 0.13103987]
Accuracy: 0.5873

Feature Column 3:
Predicted attribute values: [12 12 12 12 10]
True attribute values: [16. 12.  9. 12. 10.]
Accuracy: 0.4649

Feature Column 4:
Predicted attribute values: [16  2 16  2 16]
True attribute values: [11.  2. 16.  3. 16.]
Accuracy: 0.4856

Feature Column 5:
Predicted attribute values: [3 3 3 5 5]
True attribute values: [7. 3. 3. 5. 7.]
Accuracy: 0.7437

Feature Column 6:
Predicted attribute values: [ 3  3 12  1  1]
True attribute values: [3. 5. 3. 8. 7.]
Accuracy: 0

{'avg_discrete_accuracy': np.float64(0.657543830518259),
 'avg_continuous_accuracy': np.float64(0.6551776070702583),
 'overall_avg_accuracy': np.float64(0.6567550893689255)}